In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
HTML 파일 일괄 변환 스크립트 - 간편 설정 버전
주석만 풀면 동작이 바뀝니다!

기안일: 20210105123307 → 2021-01-05 12:33:07
보존연한: 3년 0월 → 3년
"""

import os
import re
import shutil
from pathlib import Path
from datetime import datetime
import logging

# ============================================================
# ⚙️ 설정 (여기만 수정하세요!)
# ============================================================

# 📁 처리할 디렉토리 경로
ROOT_DIRECTORY = r"C:\Users\LEEJUHWAN\Downloads\2011-01-01~2015-12-31\html\결재"

# 🧪 테스트 모드 (10개만 처리)
# TEST_MODE = True  # False로 바꾸면 전체 파일 처리
TEST_MODE = False  # 주석 풀면 전체 파일 처리

# 💾 저장 방식 선택
SAVE_TO_NEW_FOLDER = False  # True: 새 폴더에 저장 (원본 보존)
                            # False: 원본 파일 직접 수정

# 📂 새 폴더 이름 (SAVE_TO_NEW_FOLDER = True일 때만 사용)
OUTPUT_FOLDER_NAME = "결재_변환완료"

# ============================================================

# 로깅 설정
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(message)s',
    handlers=[
        logging.FileHandler('html_conversion.log', encoding='utf-8'),
        logging.StreamHandler()
    ]
)

def convert_date_format(date_str):
    """날짜 형식 변환: 20210104151446 -> 2021-01-04 15:14:46"""
    try:
        if len(date_str) != 14 or not date_str.isdigit():
            return None
        year = date_str[0:4]
        month = date_str[4:6]
        day = date_str[6:8]
        hour = date_str[8:10]
        minute = date_str[10:12]
        second = date_str[12:14]
        datetime(int(year), int(month), int(day), int(hour), int(minute), int(second))
        return f"{year}-{month}-{day} {hour}:{minute}:{second}"
    except (ValueError, IndexError):
        return None

def convert_retention_period(period_str):
    """보존연한 형식 변환: 3년 0월 -> 3년"""
    match = re.search(r'(\d+)년\s*\d+월', period_str.strip())
    if match:
        return f"{match.group(1)}년"
    return None

def process_html_content(content):
    """HTML 내용 변환"""
    original_content = content
    changes = {'date': None, 'period': None}
    
    # bg01 클래스의 모든 div 찾기
    pattern = r'<div class="bg01">\s*(.*?)\s*</div>'
    matches = list(re.finditer(pattern, content, re.DOTALL))
    
    if len(matches) < 4:
        return None, changes
    
    # 두 번째 bg01 (기안일) 처리
    second_match = matches[1]
    date_text = second_match.group(1).strip()
    converted_date = convert_date_format(date_text)
    
    if converted_date:
        old_div = second_match.group(0)
        new_div = f'<div class="bg01">{converted_date}</div>'
        content = content.replace(old_div, new_div, 1)
        changes['date'] = (date_text, converted_date)
    
    # 네 번째 bg01 (보존연한) 처리
    matches_updated = list(re.finditer(pattern, content, re.DOTALL))
    if len(matches_updated) >= 4:
        fourth_match = matches_updated[3]
        period_text = fourth_match.group(1).strip()
        converted_period = convert_retention_period(period_text)
        
        if converted_period:
            new_div = f'<div class="bg01">\n\t\t\t\t\t\t\t{converted_period}\n\t\t\t\t\t\t</div>'
            content = content[:fourth_match.start()] + new_div + content[fourth_match.end():]
            changes['period'] = (period_text, converted_period)
    
    if content == original_content:
        return None, changes
    
    return content, changes

def process_file(file_path, output_path):
    """단일 파일 처리"""
    try:
        # 파일 읽기
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
        
        # 내용 변환
        new_content, changes = process_html_content(content)
        
        if new_content is None:
            return False, changes
        
        # 파일 저장
        output_path.parent.mkdir(parents=True, exist_ok=True)
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(new_content)
        
        return True, changes
        
    except Exception as e:
        logging.error(f"❌ 오류: {file_path.name} - {str(e)}")
        return False, {}

def main():
    root_path = Path(ROOT_DIRECTORY)
    
    if not root_path.exists():
        print(f"❌ 디렉토리가 존재하지 않습니다: {ROOT_DIRECTORY}")
        return
    
    # 출력 디렉토리 설정
    if SAVE_TO_NEW_FOLDER:
        output_root = root_path.parent / OUTPUT_FOLDER_NAME
        output_root.mkdir(exist_ok=True)
        mode_text = f"새 폴더에 저장: {output_root}"
    else:
        output_root = root_path
        mode_text = "원본 파일 직접 수정"
    
    # HTML 파일 찾기
    html_files = list(root_path.rglob('*.html'))
    total_files = len(html_files)
    
    if total_files == 0:
        print("⚠️  처리할 HTML 파일이 없습니다.")
        return
    
    # 테스트 모드 적용
    if TEST_MODE:
        html_files = html_files[:10]
        process_count = len(html_files)
        test_text = f"🧪 테스트 모드: {process_count}개 파일만 처리"
    else:
        process_count = total_files
        test_text = f"✏️  전체 모드: {process_count}개 파일 처리"
    
    # 시작 메시지
    print("\n" + "="*70)
    print("HTML 파일 변환 스크립트")
    print("="*70)
    print(f"📁 원본 경로: {root_path}")
    print(f"💾 저장 방식: {mode_text}")
    print(f"📊 {test_text}")
    print("="*70)
    
    response = input("\n변환을 시작하시겠습니까? (y/n): ")
    if response.lower() != 'y':
        print("작업이 취소되었습니다.")
        return
    
    print("\n🚀 변환 작업 시작...\n")
    
    success_count = 0
    fail_count = 0
    
    for idx, file_path in enumerate(html_files, 1):
        # 출력 경로 결정
        if SAVE_TO_NEW_FOLDER:
            relative_path = file_path.relative_to(root_path)
            output_path = output_root / relative_path
        else:
            output_path = file_path
        
        # 파일 처리
        success, changes = process_file(file_path, output_path)
        
        if success:
            success_count += 1
            print(f"✅ [{idx}/{process_count}] {file_path.name}")
            if changes.get('date'):
                old, new = changes['date']
                print(f"   📅 기안일: {old} → {new}")
            if changes.get('period'):
                old, new = changes['period']
                print(f"   📋 보존연한: {old} → {new}")
        else:
            fail_count += 1
            if fail_count <= 3:  # 처음 3개만 표시
                print(f"⚠️  [{idx}/{process_count}] {file_path.name} - 변경사항 없음")
    
    # 결과 요약
    print("\n" + "="*70)
    print("📊 변환 완료!")
    print("="*70)
    print(f"처리된 파일: {process_count}개")
    print(f"✅ 성공: {success_count}개")
    print(f"⚠️  실패/변경없음: {fail_count}개")
    if SAVE_TO_NEW_FOLDER:
        print(f"\n💾 변환된 파일 위치: {output_root}")
        print(f"📁 원본 파일은 그대로 보존됩니다: {root_path}")
    if TEST_MODE:
        print(f"\n💡 테스트 모드였습니다. TEST_MODE = False로 바꾸면 전체 파일을 처리합니다.")
    print("="*70)

if __name__ == "__main__":
    main()


HTML 파일 변환 스크립트
📁 원본 경로: C:\Users\LEEJUHWAN\Downloads\2011-01-01~2015-12-31\html\결재
💾 저장 방식: 원본 파일 직접 수정
📊 ✏️  전체 모드: 6587개 파일 처리



변환을 시작하시겠습니까? (y/n):  y



🚀 변환 작업 시작...

✅ [1/6587] 20110102_시내교통비신청-김하응(0103)_2002908.html
   📅 기안일: 20110102174931 → 2011-01-02 17:49:31
   📋 보존연한: 3년 0월 → 3년
✅ [2/6587] 20110103_2010년 12월 법인 카드 사용내역 보고_2002909.html
   📅 기안일: 20110103144236 → 2011-01-03 14:42:36
   📋 보존연한: 3년 0월 → 3년
✅ [3/6587] 20110103_[LG전자 G-TIPIS 견적서 품의]_2002913.html
   📅 기안일: 20110103203124 → 2011-01-03 20:31:24
   📋 보존연한: 3년 0월 → 3년
✅ [4/6587] 20110103_더존 아이플러스 전산프로그램 보상 구매 및 유지보수 건._2002911.html
   📅 기안일: 20110103161631 → 2011-01-03 16:16:31
   📋 보존연한: 3년 0월 → 3년
✅ [5/6587] 20110103_선영프로젝트 법인카드사용내역 정산보고_2002912.html
   📅 기안일: 20110103174426 → 2011-01-03 17:44:26
   📋 보존연한: 3년 0월 → 3년
✅ [6/6587] 20110103_휴가 사용 품의 _2002910.html
   📅 기안일: 20110103154719 → 2011-01-03 15:47:19
   📋 보존연한: 3년 0월 → 3년
✅ [7/6587] 20110104_LG하우시스 PMS 권한요청_2002917.html
   📅 기안일: 20110104190355 → 2011-01-04 19:03:55
   📋 보존연한: 3년 0월 → 3년
✅ [8/6587] 20110104_[결제품의]유료 채용 공고-잡코리아_2002915.html
   📅 기안일: 20110104132952 → 2011-01-04 13:29:52
   📋 보존연한: 3년 0월 → 3년
✅ [9/